In [ ]:
import numpy as np
import time
import pickle
#import plotly.graph_objects as go

#from scipy.integrate import RK45
#from scipy.integrate import odeint
from scipy.integrate import solve_ivp
#from scipy.optimize import fsolve
#from scipy.spatial.distance import pdist
#from scipy.stats import linregress

#import matplotlib.pyplot as plt

## Input parameters

In [ ]:
### Change these parameters for each run

### Domain
# number of elements on fault
n = 200
# element length (km)
dl = 0.05

# state evolution distance (m)
dc = 0.01

In [ ]:
### Domain
# number of elements on fault
#n = 200
#n = 500
#n = 1000
# element length (km)
#dl = 0.05
#dl = 0.02
#dl = 0.01
# Fault length (km)
FL = n*dl
# Central VW length (km)
VWL = 6 
# shear wave velocity (km/s)
vs = 3 
# shear modulus (MPa)
mu=30
# radiation damping coefficient (MPa*s/m)
eta = mu/2/vs
# Normal stress (MPa)
sigma = 100
# Poisson ratio
pois = 0.25

### Rate-state
# Reference velocity (km/s)
vref = 1e-6
# reference friciton coefficient
fref = 0.6
# a
amin = 0.015   # for VW zone
amax = 0.025   # for VS zone
vspts = int(np.ceil((FL-VWL)/2/dl))   # number of elements in each side of VS zone
vwpts = int(np.ceil(VWL)/dl)          # number of elements in central VW zone
a = np.zeros(n)
a[0:vspts] = amax
a[vspts:vspts+vwpts] = amin
a[vspts+vwpts:] = amax
# b
b = 0.020
# state evolution distance (m)
#dc = 0.002
#dc = 0.004
#dc = 0.001
# (Plate) loading velocity
vl = 1e-9

### Simulation time (s)
tmax = 1e10



In [ ]:
# solve for root using bisection method
def bisection(f, interval, tol):
    """
    param f: find root for function
    param interval: within range
    param tol: root accuracy tolerance
    """

    # extract interval start and end points
    x0, x1 = interval[0], interval[1]

    # check interval can be used to solve for root
    if not validate_interval(f, x0, x1):
        return

    # iterations required to find the root within a specified error bound
    n = error_bound(x0, x1, tol)

    #counter = 1

    # iterate over error bound
    while True:

        # calculate root approximation
        root_approx = x0 + ((x1 - x0) / 2)

        # evaluate y at current estimate
        y = f(root_approx)

        # check tolerance condition
        if -tol < y < tol:
            # check that error bound actually worked
            #print(counter, n)

            # return root approximation
            return root_approx

        # check if next segment is left of bisection
        if validate_interval(f, x0, root_approx):
            x1 = root_approx
        else:
            x0 = root_approx

        # increment counter
        #counter += 1
        
def validate_interval(f, x0, x1):
    return f(x0) * f(x1) < 0

# minimum iterations to find the root within an error bound
def error_bound(a, b, err):
    n = np.log((b - a) / err) / np.log(2)
    return int(np.ceil(n))

def Veq(v, tauLock,sigma,psi,eta,a,vref):
    stress = tauLock-eta*v
    f = a*np.arcsinh(v/(2*vref)*np.exp(psi/a)) 
    strength = f * sigma
    return stress-strength

### Initial Conditions
x0 = np.zeros(2*n)  # x will be our solutions vector. Even indicies contain state variable and odd indicies contain
                    # friction coefficient
# Set initial v0 to be consistent with the initial plate loading velocity at one end (i.e. a VS zone) of the fault
tau0 = sigma*amax*np.arcsinh(vl/(2*vref)*np.exp((fref+b*np.log(vref/vl))/amax))+eta*vl
psi0 = 0.635        # state variable

vmin = 0
vmax = tau0/eta
tol = 1e-6
v0 = np.zeros(n)
for i in range(n):
    v0fun = lambda v: Veq(v,tau0,sigma,psi0,eta,a[i],vref)
    v0[i] = bisection(v0fun,[vmin,vmax],tol)
    
# Set up initial conditions for psi and f
v = np.zeros(n)
for i in range(n):
    x0[2*i] = psi0
    x0[2*i+1] = psi0+a[i]*np.log(v0[i]/vref)

In [ ]:
half_hstar= b/(np.pi*(b-amin)**2)*(mu/(1-pois))*dc/sigma    # nucleation half-length
print("The nucleation half-distance is: " + str(half_hstar))      
print("W/half_hstar: " +str(VWL/half_hstar))

## Evolution equations and stress 

In [ ]:
#kernel matrix for a flat fault with plane strain (see Seagull Ch.3)
def stress(i,j,dl,mu,pois):
    factor=mu/(2.0*np.pi*(1.0-pois))
    r1=(i-j-0.5)*dl
    r2=(i-j+0.5)*dl
    sxy=-(1/r2-1/r1)*factor
    return sxy

# Evolution equations
def cycleSystem(t,x):
    # x: even indicies: slip velocity
    # x: odd indicies: friction coefficient
    # xdot= (psidot)
    #       (fdot)
    xdot = np.zeros(2*n)
    # Calculate the velocities, then minus loading velocity  
    for i in range(n): 
        v[i] = 2*vref*np.sinh(x[2*i+1]/a[i])*np.exp(-x[2*i]/a[i])
    vd=v-vl #loading from backslip
    for i in range(n):
        dtaudot = sum(K[i,:]*vd[:])  # delta_taudot_i = K_ij * v_j
        # psidot = PSI(psi,f)
        xdot[2*i] = b*vref/dc*(np.exp((fref-x[2*i])/b)-v[i]/vref)
        # fdot = F(psi,f)
        xdot[2*i+1] = (dtaudot/sigma + eta*v[i]/(sigma*a[i]) * xdot[2*i])/(1+eta*v[i]/(sigma*a[i]))
        
    return xdot


## Build K matrix

In [ ]:
K = np.zeros([n,n])
for i in range(n):
    for j in range(n):
        K[i,j]=stress(i,j,dl,mu,pois)

## Using a scipy solver

In [ ]:
t_span=[0, tmax] #simulation time (s)

# get start time
st=time.time()
sol=solve_ivp(cycleSystem, t_span, x0, method='RK23', t_eval=None,
              dense_output=False, events=None, vectorized=True,
              args=None,max_step=1e9,rtol=1e-4)

# get the end time
et = time.time()

# elapsed time
elapsed_time=et-st
print(elapsed_time)

In [ ]:
sol

## Plot

In [ ]:
# Pick out psi and f
psi_sol = np.zeros([n,len(sol.t)])
f_sol = np.zeros([n,len(sol.t)])
for i in range(n):
    psi_sol[i] = sol.y[2*i,:]
    f_sol[i] = sol.y[2*i+1,:]

# Build a geometry for fault
faultx = np.arange(0,n,1)*dl   # km
# Time steps
tsteps = np.arange(len(sol.t))

In [ ]:
# Calculate slip velocities
v_sol = np.zeros([n,len(sol.t)])
for i in range(n):
    v_sol[i,:] = 2*vref*np.sinh(f_sol[i,:]/a[i])*np.exp(-psi_sol[i,:]/a[i])

#### Slip velocity

In [ ]:
fig = go.Figure(data=go.Heatmap(
                    z=np.log10(v_sol.transpose()),
                    x=faultx,
                    y=tsteps,
                    colorscale='viridis',
                    colorbar={'title':'log10 slip velocity',
                              'title_side':'right',
                             'orientation':'v'}
                    ))

fig.update_layout(showlegend=False,width=720,height=700,
                 title='Slip velocity',
                 xaxis_title='Fault x (km)',
                 yaxis_title = 'Time step #')

fig.show()

#### Friction coefficient 

In [ ]:
fig = go.Figure(data=go.Heatmap(
                    z=f_sol.transpose(),
                    x=faultx,
                    y=tsteps,
                    colorscale=[(0, "cyan"), (0.5, "green"), (1, "yellow")],
                    colorbar={'title':'friction coefficient',
                              'title_side':'right',
                             'orientation':'v'}
                    ))

fig.update_layout(showlegend=False,width=720,height=700,
                 title='friction coefficient',
                 xaxis_title='Fault x (km)',
                 yaxis_title = 'Time step #')

fig.show()

#### Shear stress

In [ ]:
tau_sol = sigma * f_sol

fig = go.Figure(data=go.Heatmap(
                    z=tau_sol.transpose(),
                    x=faultx,
                    y=tsteps,
                    colorscale=[(0, "cyan"), (0.5, "green"), (1, "yellow")],
                    colorbar={'title':'Shear stress',
                              'title_side':'right',
                             'orientation':'v'}
                    ))

fig.update_layout(showlegend=False,width=720,height=700,
                 title='shear stress',
                 xaxis_title='Fault x (km)',
                 yaxis_title = 'Time step #')

fig.show()


## Correlation dimension

In [ ]:
def correlation_integral(dist,r_range, N):
    '''
    dist: euclidean norm, without log being taken
    r_range: generated in logspace, but without log being taken
    N: number of total points/states of the system
    '''
     
    dist = np.log10(dist)
    r_range = np.log10(r_range)
    
    Cr = []
    for r in r_range:
        Cr.append(np.count_nonzero(dist <= r)/N**2)
    
    return np.array(Cr)

In [ ]:
allStates = sol.y.T
distances = pdist(allStates)
N = len(sol.t)

In [ ]:
rmin = np.log10(np.min(distances))
print(rmin)

In [ ]:
rmax = np.log10(np.max(distances))
print(rmax)

In [ ]:
#neighborhood radius
r = np.logspace(-4, 1, 50)

In [ ]:
C = correlation_integral(distances,r,N)
line_fit=linregress(np.log10(r),np.log10(C))
print(line_fit)

In [ ]:
ax = plt.figure().add_subplot()
ax.plot(np.log10(r),np.log10(C),'bo')
ax.plot(np.log10(r),line_fit.slope*np.log10(r)+line_fit.intercept,color='black')

## Save solution

In [ ]:
save_dir = "/home/users/axelwang/pyCycle/results/"
file_name = "n200dlP05dcP01.pickle"

with open(save_dir+file_name, 'wb') as handle:
    pickle.dump(sol, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
## Load saved pickle file

with open(save_dir+file_name, 'rb') as handle:
    b = pickle.load(handle)